# proto - 解决摘要技术问题的最小原型

初步结论：

- 问题出在，默认实现嵌入的第一个有时不是匹配文档
- 使用底层api，设置嵌入的top_k，更大覆盖面，让后续llm去挑选可用的
- 底层api的llm实现，命中率更高，但是依赖llm，qwen2不稳定，另外就是慢

因此，下一步，使用top_k，增加覆盖面，减少找不到的情况


## 准备

In [1]:
%%time

INDEX_PATH="proto-index"
DATA_PATH="proto-data"

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 9.3 µs


In [2]:
%%time
!rm -rf $INDEX_PATH

CPU times: user 2.52 ms, sys: 3.85 ms, total: 6.37 ms
Wall time: 109 ms


In [3]:
%%time

from llama_index.core import SimpleDirectoryReader

documents=SimpleDirectoryReader(input_dir="./proto-data").load_data(num_workers=4)
for document in documents:
    document.doc_id=document.metadata['file_name']

CPU times: user 2.67 s, sys: 357 ms, total: 3.03 s
Wall time: 5.58 s


In [4]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.52 ms, sys: 0 ns, total: 1.52 ms
Wall time: 1.16 ms


In [5]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

# Settings.llm=get_llm("gpt-3.5-turbo")
Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 663 ms, sys: 24 ms, total: 687 ms
Wall time: 687 ms


In [6]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex

splitter = SentenceSplitter(chunk_size=1024)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/3 [00:00<?, ?it/s]

current doc id: 伊犁条约.txt
current doc id: 张之洞.txt
current doc id: 珍宝岛事件.txt


Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
%%time

doc_summary_index.storage_context.persist("proto-index")

CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 11.3 ms


## 加载索引

In [8]:
%%time

from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="proto-index")
doc_summary_index = load_index_from_storage(storage_context)

CPU times: user 21.1 ms, sys: 0 ns, total: 21.1 ms
Wall time: 20.5 ms


## 查询

### 高级查询 - 出现检索不到的问题

In [9]:
%%time

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

CPU times: user 217 µs, sys: 0 ns, total: 217 µs
Wall time: 220 µs


In [10]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    use_async=True,
    streaming=True,
)

response = query_engine.query("张之洞是谁扶植起来的，为啥要扶植他？")
response.print_response_stream()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.0.72:11435
DEBUG:urllib3.connectionpool:http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': 'Context information from multiple sources is below.\n---------------------\nfile_path: /root/notebook/my-jupyter-notebook/llm/llama_index/document-summary-index/proto-data/张之洞.txt\n\n张之洞（1837年9月2日—1909年10月4日）。字孝达，一字香涛，号香岩，又号壶公、无竞居士，晚年自号抱冰，人称“张香帅”，直隶南皮县（今河北南皮）人，晚清重臣。道光十七

In [10]:
response = query_engine.query("伊犁条约是由谁签订的？")
response.print_response_stream()

《伊犁条约》是由清朝与俄国签定的条约。在不同阶段，清政府派出了崇厚和曾纪泽赴俄谈判并最终签订该条约。其中，崇厚在沙俄胁迫下擅自签订了《里瓦几亚条约》，而《中俄改订条约》即《伊犁条约》则是由曾纪泽在光绪七年（1881年）与俄国签订的。

In [11]:
response = query_engine.query("为啥中国和苏联对珍宝岛的归属有争议？")
response.print_response_stream()

关于珍宝岛的归属争议，并不在提供的张之洞信息中提及，因此无法根据这些资料来解答这个问题。张之洞是晚清重臣，其活动时间与珍宝岛事件无关。珍宝岛事件发生在20世纪60年代末至70年代初的中苏边界冲突，涉及的是中华人民共和国和苏联（现俄罗斯）之间的领土争议。

In [13]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=1,
)

retrieved_nodes = retriever.retrieve("为啥中国和苏联对珍宝岛的归属有争议？")

len(retrieved_nodes)

CPU times: user 3.84 ms, sys: 1.45 ms, total: 5.29 ms
Wall time: 111 ms


2

In [14]:
retrieved_nodes[0].node.get_text()

'张之洞（1837年9月2日—1909年10月4日）。字孝达，一字香涛，号香岩，又号壶公、无竞居士，晚年自号抱冰，人称“张香帅”，直隶南皮县（今河北南皮）人，晚清重臣。道光十七年八月初三生于贵州。咸丰二年（1852年）15岁中顺天乡试解元，同治二年（1863年）中进士第三名探花，据说是因为慈禧太后喜欢张之洞的文章，因此破例提拔，授翰林院编修。历任教习、侍读、侍讲、内阁学士、山西巡抚、两广总督、湖广总督、两江总督（多次署理，从未实授）、军机大臣等职，官至体仁阁大学士。\n\n张之洞是慈禧扶植起来以抗衡湘军和淮军势力的重要人物，又是清流派重要成员，一生在清廷权力的中心周旋，试图改革中国而又不动政制。他对西方政治、经济、科学技术了解甚深，但与以康有为、梁启超为代表的激进改良派持有不同的主张。他对革命党人采取镇压态度。\n\n而另一方面，他又是继曾国藩、李鸿章后，洋务运动的领袖人物，推动清朝军工改革，因此在清亡后被遗老们视为清朝灭亡的“祸首”。张之洞为官清廉，家族只有薄田若干，即使死后亦不增一亩，但为支撑工厂庞大开支而滥权募资（如抽签捐款），得罪了当地富商。因此，张之洞虽为晚清政治界、思想界、经济界和文学界举足轻重的人物，但树敌颇多，死后毁誉不一，甚至毁多誉少。特别是张之洞对李鸿章似有成见，尤其不满其身后谥为文忠（似乎更不满其师曾国藩之文正），不料自己死后是个三等“文襄”。\n\n去世之际，《天津大公报》评论说：观其一生，“张相国毫无宗旨，毫无政见，随波逐流，媚主以求荣之人也。”\n\n梁启超直言张之洞爱名爱身甚于爱国：“除弊之事，最易犯众忌而触众怒，故全躯保位惜名之人，每不肯为之……张之洞且尝与余言，言废八股为变法第一事矣。而不闻其上折请废之者，盖恐触数百翰林、数千进士、数万举人、教十万秀才、数百万童生之怒，惧其合力以谤己而排挤己也。'

In [16]:
retrieved_nodes[1].node.get_text()

'”\n\n梁启超直言张之洞爱名爱身甚于爱国：“除弊之事，最易犯众忌而触众怒，故全躯保位惜名之人，每不肯为之……张之洞且尝与余言，言废八股为变法第一事矣。而不闻其上折请废之者，盖恐触数百翰林、数千进士、数万举人、教十万秀才、数百万童生之怒，惧其合力以谤己而排挤己也。今夫所谓爱国之士，苟其事有利于国者，则虽败己之身，裂己之名，犹当为之。今既自谓爱国矣，又复爱身焉，又复爱名焉，及至三者不可得兼，则舍国而爱身名，至二者不可得兼，又将舍名而爱身……乃浮华之人也。[5]”但他主政湖广多年，百姓口碑颇佳，到处都是自强自诩，包括汉阳铁厂和江滩的大场面[6]。而且，他在湖北的建树为日后辛亥革命作出贡献。\n\n孙中山曾称“张之洞是不言革命之大革命家”[7]。\n\n袁世凯曾盛赞张之洞学问。但却又消遣其不够务实：“张中堂是讲学问的；我是不讲学问的，我是讲办事的。”张之洞幕僚辜鸿铭听闻，愤而斥之[8]。\n\n郑孝胥曾说“南皮（张之洞）有学无术，项城（袁世凯）不学有术，西林（岑春煊）无学无术，惟端公（端方）有学有术。”张之洞听说之后，笑着表示：“我是无术，也无学。不过比项城跟西林多认识了几个字。袁项城何止有术，根本是多术！至于端方，只是整天搜集假字画跟假碑文，这样也叫做有学？根本是郑孝胥在巴结。”消遣了郑、端、袁三人。[9]日本探讨近代史1990年名著辛亥革命，认为革命发生在湖北，与张生前任湖北都督的长期包庇革命党人大有关系。\n\n清政府驻日本学监、汉军旗李孺（原名李宝巽，汉军旗进士胡俊章侄女婿）在其《挽张之洞联》中评论道：“魂魄有知，犹登岘山，羊太傅未忘旧治；庶政躬亲，不舍昼夜，司马公同此忠清”。'

In [17]:
retrieved_nodes = retriever.retrieve("伊犁条约是由谁签订的？")

len(retrieved_nodes)

1

In [18]:
retrieved_nodes[0].node.get_text()

'《伊犁条约》（又称《中俄改订条约》或《圣彼得堡条约》）是清朝与俄国签定有关归还新疆伊犁地区的条约，于光绪七年正月二十六日（1881年2月24日）在圣彼得堡签定。中方条约文本原藏于中华民国外交部，现典藏于台北外双溪国立故宫博物院。\n\n同治十年（1871年），沙俄趁浩罕汗国领导人阿古柏在天山南路宣布东突厥斯坦独立之际，出兵新疆伊犁地区；清政府多次交涉无果。\n\n光绪三年（1877年），清政府平定新疆；次年派崇厚赴俄谈判收回伊犁问题。\n\n光绪五年八月十七日（1879年10月2日），崇厚在沙俄胁迫下擅自签定《里瓦几亚条约》；按约中国仅收回伊犁河上游谷地，失去伊犁西部、南部和南、北疆边境土地甚多，此外还偿付兵费五百万卢布，朝野纷纷反对，清廷未许。\n\n光绪六年（1880年），改派曾纪泽赴俄修订条约；光绪七年正月二十六日（1881年2月24日），签定《中俄改订条约》，即《伊犁条约》，因于俄国国都圣彼得堡签订又称《圣彼得堡条约》。\n\n《改订条约》下签订有：\n\n子约一：伊犁界约 1882年8月6日 勘分霍尔果斯河段。\n子约二：喀什噶尔界约（东北界约） 1882年12月7日 勘分喀什噶尔东北段，将边界由天山主脊南推。\n子约三：科布多塔尔巴哈台界约（科塔界约） 1883年8月12日：勘分科布多至塔城段。\n子约四：塔尔巴哈台西南界约 1883年10月3日 勘分塔城西南段，将边界由天山主脊南推。\n子约五：续勘喀什噶尔界约 1884年6月3日 勘分喀什噶尔西北段，即著名的乌孜别里山口“俄国边界转向西南，中国边界一直向南”。'

### 底层api, 嵌入 - 设置top_k可以缓解问题

In [19]:
retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=3,
)

retrieved_nodes = retriever.retrieve("为啥中国和苏联对珍宝岛的归属有争议？")

len(retrieved_nodes)

5

In [27]:
retrieved_nodes[3].node.get_text()

'珍宝岛事件（中方称为珍宝岛自卫反击战）是指中华人民共和国和苏联因黑龙江流域的珍宝岛的归属问题于1969年3月间在岛上发生的武装冲突。事件以中方胜利告终，中方在事件后实际控制珍宝岛。中苏边界冲突事件致使中苏关系进一步恶化。珍宝岛一役后，苏联高层曾多次商讨要对中华人民共和国进行报复，并明确发出先发制人和核打击的威胁，中方作出相应舆论回击。苏联更是从周边围堵和威胁当时的中华人民共和国。这使得珍宝岛事件成为继古巴导弹危机后冷战期间又一次爆发全面核战争的重大危机。珍宝岛事件以及苏联的核威胁，促使毛泽东等中共领导层考虑缓和与美国和西方阵营的关系，中美关系此后迅速破冰[7][8][9]。\n\n背景：\n\n根据1860年清政府和俄罗斯帝国签署的《中俄北京条约》，中俄以乌苏里江为界。由于该岛位于界河之上，归属在整个二十世纪没有定论，中国和俄国（苏联）都曾声称拥有该岛主权。中方认为，据《中俄北京条约》，边界在乌苏里江主航道上，而该岛明显在主航道以西（中国一侧）故归属中国。\n\n第二次世界大战末期八月风暴行动之后，从1947年开始苏联红军在此巡逻。从1960年代初开始，中苏正式交恶。1964年中苏边境谈判中，苏方准备将该岛给予中方，但由于当时的中苏关系日趋恶化而中止。\n\n1966年，文化大革命爆发，毛泽东号召“反苏修”[10]，中国共产党批判苏联为“苏修叛徒集团”[11][12]。1967年发生莫斯科红场事件，进一步导致中苏关系恶化。\n\n经过：\n\n1969年3月2日，中苏两国边防部队在中苏边界的珍宝岛爆发冲突，此次冲突被中方称为“珍宝岛保卫战”或“珍宝岛自卫反击战”。根据当时的文字记录，这场冲突起因是苏军发现中方有人越境进入苏方控制区域，遂派出边防巡逻队试图驱逐对方，却不料自己被引入伏击圈，随后遭到中方的猛烈开火，导致数十名苏军士兵被打死打伤。'

In [30]:
%%time

from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    # streaming=True,
)

response = query_engine.query("为啥中国和苏联对珍宝岛的归属有争议？")
print(response)

中国和苏联（现在的俄罗斯）对珍宝岛的归属存在争议，主要基于历史条约的解释、地缘政治因素以及双方关系的变化。根据1860年清政府与俄罗斯帝国签署的《中俄北京条约》，两国以乌苏里江为界。由于珍宝岛位于乌苏里江之上，其具体位置引发了争议。

中国认为，按照条约规定，边界应在乌苏里江主航道上，而珍宝岛明显位于主航道以西（即中国一侧），因此应属于中国。然而，苏联（俄罗斯）则可能基于不同的解释或历史背景主张岛屿主权。在第二次世界大战末期的八月风暴行动之后，从1947年开始，苏联红军开始在该地区巡逻。

随着中苏关系的恶化，特别是在1960年代初开始的正式交恶以及随后的文化大革命爆发、毛泽东号召“反苏修”和莫斯科红场事件等事件的发生，导致了中苏关系进一步紧张。这些政治因素加剧了双方对珍宝岛主权的争议，并最终引发了珍宝岛冲突。

综上所述，中国和苏联（俄罗斯）对珍宝岛归属有争议的原因包括历史条约解释、地缘政治考量以及两国间的政治关系变化。
CPU times: user 74 ms, sys: 6.77 ms, total: 80.8 ms
Wall time: 15 s


In [33]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=3,
    # streaming=True,
)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,streaming=True
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    # streaming=True,
)

response = query_engine.query("为啥中国和苏联对珍宝岛的归属有争议？")
response.print_response_stream()

关于珍宝岛（乌苏里岛）的归属争议，主要源于历史上的条约解释、地缘政治因素以及中苏两国之间的复杂关系。1960年，根据《中俄北京条约》，中俄以乌苏里江为界。然而，珍宝岛位于乌苏里江之上，其具体位置在主航道的界定上存在不同的理解。

中国认为，《中俄北京条约》规定边界在乌苏里江主航道上，而珍宝岛明显位于主航道以西（即中国一侧），因此应归中国所有。这一观点基于对条约中“主航道”的解释和地理位置的实际观察。

苏联则从1947年开始在该地区驻军巡逻，并坚持其对该岛屿的主权主张。随着1960年代初中苏关系恶化，以及1964年两国边境谈判中的相关讨论，珍宝岛的归属问题变得更加敏感和复杂。

1969年的“珍宝岛事件”中，中国与苏联在该岛上发生了武装冲突，这一事件加剧了双方之间的紧张局势。尽管此后珍宝岛的实际控制权发生了变化，但具体的法律依据、国际承认过程以及最终解决方式较为复杂，并且涉及历史背景和地缘政治的多方面考量。

综上所述，珍宝岛归属争议源于对条约解释的不同理解、地缘战略考虑以及中苏两国关系的历史演变。CPU times: user 375 ms, sys: 48 ms, total: 423 ms
Wall time: 14.9 s


### 底层api，LLM - 慢、不稳定，但是结果是正确的

In [68]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
)


retriever = DocumentSummaryIndexLLMRetriever(
    doc_summary_index,
    # choice_select_prompt=None,
    choice_batch_size=20,
    # choice_top_k=10,
    # format_node_batch_fn=None,
    # parse_choice_select_answer_fn=None,
)

retrieved_nodes = retriever.retrieve("为啥中国和苏联对珍宝岛的归属有争议？")
len(retrieved_nodes)

CPU times: user 6.81 ms, sys: 0 ns, total: 6.81 ms
Wall time: 2.97 s


2

In [66]:
retrieved_nodes[0].score

10.0

In [56]:
retrieved_nodes[0].node.get_text()

'珍宝岛事件（中方称为珍宝岛自卫反击战）是指中华人民共和国和苏联因黑龙江流域的珍宝岛的归属问题于1969年3月间在岛上发生的武装冲突。事件以中方胜利告终，中方在事件后实际控制珍宝岛。中苏边界冲突事件致使中苏关系进一步恶化。珍宝岛一役后，苏联高层曾多次商讨要对中华人民共和国进行报复，并明确发出先发制人和核打击的威胁，中方作出相应舆论回击。苏联更是从周边围堵和威胁当时的中华人民共和国。这使得珍宝岛事件成为继古巴导弹危机后冷战期间又一次爆发全面核战争的重大危机。珍宝岛事件以及苏联的核威胁，促使毛泽东等中共领导层考虑缓和与美国和西方阵营的关系，中美关系此后迅速破冰[7][8][9]。\n\n背景：\n\n根据1860年清政府和俄罗斯帝国签署的《中俄北京条约》，中俄以乌苏里江为界。由于该岛位于界河之上，归属在整个二十世纪没有定论，中国和俄国（苏联）都曾声称拥有该岛主权。中方认为，据《中俄北京条约》，边界在乌苏里江主航道上，而该岛明显在主航道以西（中国一侧）故归属中国。\n\n第二次世界大战末期八月风暴行动之后，从1947年开始苏联红军在此巡逻。从1960年代初开始，中苏正式交恶。1964年中苏边境谈判中，苏方准备将该岛给予中方，但由于当时的中苏关系日趋恶化而中止。\n\n1966年，文化大革命爆发，毛泽东号召“反苏修”[10]，中国共产党批判苏联为“苏修叛徒集团”[11][12]。1967年发生莫斯科红场事件，进一步导致中苏关系恶化。\n\n经过：\n\n1969年3月2日，中苏两国边防部队在中苏边界的珍宝岛爆发冲突，此次冲突被中方称为“珍宝岛保卫战”或“珍宝岛自卫反击战”。根据当时的文字记录，这场冲突起因是苏军发现中方有人越境进入苏方控制区域，遂派出边防巡逻队试图驱逐对方，却不料自己被引入伏击圈，随后遭到中方的猛烈开火，导致数十名苏军士兵被打死打伤。'

In [41]:
retrieved_nodes[1].score

10.0

In [39]:
retrieved_nodes[1].node.get_text()

'根据当时的文字记录，这场冲突起因是苏军发现中方有人越境进入苏方控制区域，遂派出边防巡逻队试图驱逐对方，却不料自己被引入伏击圈，随后遭到中方的猛烈开火，导致数十名苏军士兵被打死打伤。3月15日和17日，双方再次发生武装冲突，中方宣称是对方蓄意挑衅并先发制人，而苏方则认为自己是被迫自卫。这场冲突持续了大约一天时间，双方都有重火力参与，甚至有报道称中方的步兵团以及大炮、迫击炮和坦克支援参战。\n\n在随后的战斗中，苏联增派了大量正规军力量，并动用了BM-21火箭炮等重型武器。中方则使用了包括反坦克炮和无后座力炮在内的轻型火力和岸上炮火。双方都指责对方先发制人，引发冲突。这场冲突最终以苏方撤退和中方向江心岛投下大量地雷并封锁边界线结束。\n\n在这场冲突中，苏联的T-62坦克履带被地雷击毁，其乘员弃车逃生。苏联边防总队长列昂诺夫上校在3月17日的战斗中阵亡。中方则报告称打捞出了一辆从江面上被毁的苏军T-62坦克，并将它修复并转移到了北京的人民革命军事博物馆作为中国人民解放军自制坦克的原型。\n\n这场冲突引起了国际社会的广泛关注，尤其是美国和苏联之间的核危机。根据当时的记录，苏联官方在1978年否认了对于中国进行核打击的意图。珍宝岛冲突最终导致双方各有伤亡，中方声称击毙或打伤苏军230余人，毁伤19辆坦克装甲车辆，而苏方公布的数字则为152人。这场冲突被认为是中国边界防御的一次胜利，同时也是中苏两国关系紧张的一个标志性事件。'